In [9]:
import csv
import random
import math
def loadCsv(filename):
    lines=csv.reader(open(filename,"rt"))
    dataset=list(lines)
    for i in range(len(dataset)):
        dataset[i]=[float(x)for x in dataset[i]]
    return dataset
def splitDataset(dataset,splitRatio):
    trainSize=int(len(dataset)*splitRatio)
    trainSet=[]
    copy=list(dataset)
    while len(trainSet)<trainSize:
        index=random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return[trainSet,copy]
def separateByclass(dataset):
    separated={}
    for i in range(len(dataset)):
        vector=dataset[i]
        if (vector[-1]not in separated):
            separated[vector[-1]]=[]
        separated[vector[-1]].append(vector)
    return separated
def mean(numbers):
    return sum(numbers)/float(len(numbers))
def stdev(numbers):
    avg=mean(numbers)
    variance=sum([pow(x-avg,2)for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)
def summarize(dataset):
    summaries=[(mean(attribute),stdev(attribute))for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries
def summarizeByclass(dataset):
    separated=separateByclass(dataset)
    print(len(separated))
    summaries={}
    for classvalue,instances in separated.items():
        summaries[classvalue]=summarize(instances)
    print(summaries)
    return summaries
def calculateprobability(x,mean,stdev):
    exponent=math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1/(math.sqrt(2*math.pi)*stdev))*exponent
def calculateClassProbabilities(summaries,inputvector):
    probabilities={}
    for classvalue,classsummaries in summaries.items():
        probabilities[classvalue]=1
        for i in range (len(classsummaries)):
            mean,stdev=classsummaries[i]
            x=inputvector[i]
            probabilities[classvalue]*=calculateprobability(x,mean,stdev)
    return probabilities
def predict(summmaries,inputVector):
    probabilities=calculateClassProbabilities(summaries,inputVector)
    bestLabel,bestProb=None,-1
    for classValue,probability in probabilities.items():
        if bestLabel is None or probability>bestProb:
            bestProb=probability
            bestLabel=classValue
    return bestLabel
def getPredictions(summaries,testSet):
    predictions=[]
    for i in range(len(testSet)):
        result=predict(summaries,testSet[i])
        predictions.append(result)
    return predictions
def getAccuracy(testSet,predictions):
    correct=0
    for i in range(len(testSet)):
        if testSet[i][-1]==predictions[i]:
            correct+=1
    return(correct/float(len(testSet)))*100.0
filename="DBetes.csv"
splitRatio=0.70
dataset=loadCsv(filename)
trainingSet,testSet=splitDataset(dataset,splitRatio)
print('Split{0} rows into train={1} and test={2} rows'.format(len(dataset),len(trainingSet),len(testSet)))
summaries=summarizeByclass(trainingSet)
predictions=getPredictions(summaries,testSet)
accuracy=getAccuracy(testSet,predictions)
print('Accuracy:{0}%'.format(accuracy))

Split250 rows into train=175 and test=75 rows
2
{0.0: [(3.2636363636363637, 2.8467824191154523), (109.36363636363636, 26.359799554000897), (68.06363636363636, 19.57637467385477), (18.9, 15.34313939862062), (57.49090909090909, 112.59440446397494)], 1.0: [(5.6923076923076925, 3.9048170448621726), (141.3846153846154, 30.899884864112114), (69.9076923076923, 20.512742287511102), (21.70769230769231, 17.0355538845629), (99.61538461538461, 151.75079203949937)]}
Accuracy:69.33333333333334%
